In [1]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np, sklearn, seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
df= pd.read_csv('data/citylearn_challenge_2023_phase_3_3citylearn_challenge_2023_phase_3_3/Building_1.csv', parse_dates=True).set_index('timestamp')
df.head()

# Scaling the data

In [ ]:
from sklearn import preprocessing

In [ ]:
scaler= preprocessing.MinMaxScaler()
# only scaling the numerical values not day/month
scaled_data=scaler.fit_transform(df.drop(columns=['day','month']))

In [ ]:
type(scaled_data)

adding the day/month again, and constructing a df

In [ ]:
scaled_df= pd.concat( [ pd.DataFrame(scaled_data, index=df.index , columns=df.columns[:-2]) , df.iloc[:,-2:] ] 
                     , axis=1 )
scaled_df.head()

# Creating training and testing and validation

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_hidden,y_train,y_hidden= train_test_split( scaled_df.drop(columns='meter_reading') ,
                                                    scaled_df.meter_reading,
                                                    test_size=0.2,
                                                    shuffle=False )

In [ ]:
assert len(y_train) ==len(x_train)

we are good to go now

In [ ]:
x_test,x_val,y_test,y_val= train_test_split(x_hidden,y_hidden,
                                             test_size=0.5,
                                             shuffle=False)

## we have now:

1.   x_train & y_train
2.   x_test & y_test
3.   x_val & y_val



# Creating simple models

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [ ]:
lasso_model= Lasso( alpha=0.3 )
lasso_model.fit(x_train , y_train )
lasso_pred= lasso_model.predict( x_test )

print( mean_squared_error( y_test , lasso_pred) )

## Clearly we can see that linear models only use the mean as the predicted value

In [ ]:
_, axs=plt.subplots(2,1, figsize=(15,15))
sns.lineplot( range(len(x_test)) , lasso_pred, ax=axs[0] )
sns.lineplot( range(len(x_test)) , y_test, ax=axs[1] )

# let's try more soffisticated models

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forrest_model= RandomForestRegressor()
forrest_model.fit(x_train , y_train )
forrest_pred= forrest_model.predict( x_test )

print( mean_squared_error( y_test , forrest_pred) )

## Let's try to plot it


In [ ]:
_, axs=plt.subplots(2,1, figsize=(15,15))
sns.lineplot( range(len(x_test)) , forrest_pred, ax=axs[0] )
sns.lineplot( range(len(x_test)) , y_test, ax=axs[1] )

we can see that the output is not linear, but still not effecient enough

# last try with powerful model


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gradient_model= GradientBoostingRegressor( n_estimators=1000)

gradient_model.fit(x_train , y_train )
gradient_pred= gradient_model.predict( x_test )

print( mean_squared_error( y_test , gradient_pred) )

_, axs=plt.subplots(2,1, figsize=(15,15))
sns.lineplot( range(len(x_test)) , gradient_pred, ax=axs[0] )
sns.lineplot( range(len(x_test)) , y_test, ax=axs[1] )

# and we can see now that we need sequential models

## let's save the data so that we work with a standard version

In [ ]:
list_of_df= [x_train , y_train,
            x_test , y_test,
            x_val , y_val]

list_of_df_names= ['x_train' , 'y_train',
                    'x_test' , 'y_test',
                    'x_val' , 'y_val']

for i, frame in enumerate(list_of_df):
    frame.to_csv( f'content/{list_of_df_names[i]}.csv')